In [13]:
#seperate and organize Combo model prediction output into binary_pr masks and pr masks
import os
import shutil

def sort_files_by_pattern(source_dir, binary_pattern, general_pattern, binary_dir, general_dir):
    # Ensure destination folders exist
    os.makedirs(binary_dir, exist_ok=True)
    os.makedirs(general_dir, exist_ok=True)

    for filename in os.listdir(source_dir):
        source_path = os.path.join(source_dir, filename)

        if os.path.isfile(source_path):
            if binary_pattern in filename:
                shutil.move(source_path, os.path.join(binary_dir, filename))
                print(f"Moved to binary folder: {filename}")
            elif general_pattern in filename:
                shutil.move(source_path, os.path.join(general_dir, filename))
                print(f"Moved to general folder: {filename}")

# Example usage
source_directory = r"input to Combo model prediction output arrays"
binary_pr_mask_directory = r"path to output for Combo model binary arrays"
pr_mask_directory = r"path to output for Combo model pr array"

sort_files_by_pattern(
    source_dir=source_directory,
    binary_pattern="binary_pr_mask",
    general_pattern="pr_mask",
    binary_dir=binary_pr_mask_directory,
    general_dir=pr_mask_directory
)


Moved to binary folder: TG_1_0_0_binary_pr_mask.npy
Moved to general folder: TG_1_0_0_pr_mask.npy
Moved to binary folder: TG_1_0_1000_binary_pr_mask.npy
Moved to general folder: TG_1_0_1000_pr_mask.npy
Moved to binary folder: TG_1_0_1500_binary_pr_mask.npy
Moved to general folder: TG_1_0_1500_pr_mask.npy
Moved to binary folder: TG_1_0_2000_binary_pr_mask.npy
Moved to general folder: TG_1_0_2000_pr_mask.npy
Moved to binary folder: TG_1_0_2500_binary_pr_mask.npy
Moved to general folder: TG_1_0_2500_pr_mask.npy
Moved to binary folder: TG_1_0_3000_binary_pr_mask.npy
Moved to general folder: TG_1_0_3000_pr_mask.npy
Moved to binary folder: TG_1_0_3500_binary_pr_mask.npy
Moved to general folder: TG_1_0_3500_pr_mask.npy
Moved to binary folder: TG_1_0_4000_binary_pr_mask.npy
Moved to general folder: TG_1_0_4000_pr_mask.npy
Moved to binary folder: TG_1_0_4500_binary_pr_mask.npy
Moved to general folder: TG_1_0_4500_pr_mask.npy
Moved to binary folder: TG_1_0_5000_binary_pr_mask.npy
Moved to genera

In [5]:
#check array size
import numpy as np

def read_array_and_print_dimensions(file_path):
    """
    Reads a NumPy array file and prints its dimensions.

    Parameters:
        file_path (str): The path to the .npy file.

    Returns:
        None
    """
    try:
        # Load the NumPy array from the file
        array = np.load(file_path)
        
        # Print the dimensions of the array
        print(f"Array dimensions: {array.shape}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
if __name__ == "__main__":
    # Replace 'example.npy' with the path to your .npy file
    file_path = r'path to original output arrays'
    read_array_and_print_dimensions(file_path)


Array dimensions: (5512, 7012)


In [14]:
#resize arrays
import os
import numpy as np
from skimage.transform import resize

def resize_npy_files(input_dir, output_dir, target_shape=(500, 500)):
    # Check if output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over files in the input directory
    for filename in os.listdir(input_dir):
        # Construct the full file path
        input_file_path = os.path.join(input_dir, filename)

        # Check if it is a .npy file
        if filename.endswith('.npy'):
            try:
                # Load the numpy array from the .npy file
                array_512x512 = np.load(input_file_path)

                # Resize the array to the target shape
                array_resized = resize(array_512x512, target_shape, anti_aliasing=True)

                # Construct the output file path
                output_file_path = os.path.join(output_dir, filename)

                # Save the resized numpy array to the output directory
                np.save(output_file_path, array_resized)

                print(f"Resized {filename} and saved to {output_file_path}")

            except Exception as e:
                print(f"Error resizing {filename}: {e}")

# Example usage
input_directory = r"path to original binary arrays"   # Change this to your input directory
output_directory = r"path to output of resized binary arrays"    # Change this to your output directory

resize_npy_files(input_directory, output_directory)


Resized TG_1_0_0_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_0_binary_pr_mask.npy
Resized TG_1_0_1000_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_1000_binary_pr_mask.npy
Resized TG_1_0_1500_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_1500_binary_pr_mask.npy
Resized TG_1_0_2000_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_2000_binary_pr_mask.npy
Resized TG_1_0_2500_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_2500_binary_pr_mask.npy
Resized TG_1_0_3000_binary_pr_mask.npy and saved to C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\resized\TG_1_0_3000_binary_pr_mask.npy
Resized TG_1_0_3500_binary_p

In [15]:
#combine resized arrays into one array
import numpy as np
import os
from typing import List, Tuple

def parse_coordinates(filename: str) -> Tuple[int, int]:
    """
    Extracts the top-left coordinates from the filename.
    Assumes the filename format is "DRG_4_SegTrans_x_y_binary_pr_mask.npy".
    """
    base_name = os.path.splitext(filename)[0]
    parts = base_name.split('_')
    x, y = map(int, parts[-5:-3])  # Extract x and y from the appropriate positions
    return x, y

def merge_arrays(file_paths: List[str]) -> np.ndarray:
    """
    Merges binary numpy arrays based on their top-left coordinates.

    Args:
        file_paths (List[str]): List of file paths to the binary numpy arrays.

    Returns:
        np.ndarray: The merged numpy array.
    """
    # Parse all coordinates and arrays
    arrays = []
    coordinates = []
    for file_path in file_paths:
        coords = parse_coordinates(os.path.basename(file_path))
        array = np.load(file_path)
        arrays.append((coords, array))
        coordinates.append(coords)

    # Determine the size of the final merged array
    min_x = min(coord[0] for coord in coordinates)
    min_y = min(coord[1] for coord in coordinates)
    max_x = max(coord[0] + arrays[i][1].shape[0] for i, coord in enumerate(coordinates))
    max_y = max(coord[1] + arrays[i][1].shape[1] for i, coord in enumerate(coordinates))

    # Create the merged array filled with zeros
    merged_array = np.zeros((max_x - min_x, max_y - min_y), dtype=np.uint8)

    # Place each array into the merged array
    for (x, y), array in arrays:
        x_offset = x - min_x
        y_offset = y - min_y
        merged_array[x_offset:x_offset + array.shape[0], y_offset:y_offset + array.shape[1]] = array

    return merged_array

# Example usage
if __name__ == "__main__":
    # Replace with the directory containing your .npy files
    input_directory = r"path to resized binary arrays" 

    # Specify the output directory
    output_directory = r"path to output for final combined array"
    os.makedirs(output_directory, exist_ok=True)

    # Get all .npy files in the input directory
    file_paths = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith(".npy")]

    # Merge all arrays
    merged_array = merge_arrays(file_paths)

    # Save the merged array in the output directory
    output_path = os.path.join(output_directory, "Tname of output directory")
    np.save(output_path, merged_array)
    print(f"Merged array saved as '{output_path}'")


Merged array saved as 'C:\Users\Mouse\Desktop\segmentation_model_outputs\HE_new\0.8_TG1\binary\finalarrayresized\TG1_combo_HE_0.8'
